<a href="https://colab.research.google.com/github/Randasabag/Optimisez-la-gestion-des-donn-es-d-une-boutique-avec-R-ou-Python/blob/main/P5_alsabbagh_Optimisation_donnees_boutique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# OPTIMISATION DE LA GESTION DES DONNEES D'UNE BOUTIQUE AVEC PYTHON 

## <font color='indianred'> BOTTLENECK, MARCHAND DE VIN TRÈS PRESTIGIEUX</font>

# Mission :
<ol>
    <li>Rapprochement de deux exports : </li>
   <ul>
        <li> un <b>export de l’ERP</b> contenant les références produit, leur prix de vente et leur état de stock</li>
    <li> un <b>export d’une table de l’outil de CMS</b> contenant les informations des produits commercialisés en ligne (nom, description, nombre de ventes...)</li>
    </ul>
  
  L’export issu de la boutique en ligne contient le nombre de ventes pour chaque produit depuis sa mise en ligne, il ne permet pas d’analyser l'évolution des ventes dans le temps.
    
Un tableau Excel a été créé  afin d’établir le lien entre la référence du produit dans l’ERP (product_id) et la référence du même produit dans la base de la boutique en ligne (SKU). 

  <li> Après le rapprochement effectué, il faut obtenir : 
       <ul><li> le chiffre d’affaires par produit</li>
           <li> le total du chiffre d’affaires réalisé en ligne.</li>
      </ul>    
  </li> <br>

       
   <li> Il y a des erreurs de saisie dans certains prix des produits. Il faudrait effectuer une analyse sur cette variable afin de détecter d’éventuelles valeurs aberrantes, de les lister et d’en faire une représentation graphique pour plus de lisibilité.</li><br>

   <li> Une colonne a été mal nommée dans le fichier Excel. En fait, la colonne id_web dans le fichier correspond au SKU des produits dans la boutique en ligne.</li>   
 </ol>


# Sommaire :
**Partie 1 : Importation des données**
 - <a href="#C1">Importation des librairies</a>
 - <a href="#C2">Importation des jeux de données</a>
 
**Partie 2 : Analyse des jeux de données**
 - <a href="#C3">Affichage du nombre de lignes et colonnes des jeux de données </a>
 - <a href="#C4">Analyse des valeurs manquantes</a>
 - <a href="#C5">Analyse des types de données</a>
 
**Partie 3 : Nettoyage du jeu de données**
 - <a href="#C6">Unicité de la clé primaire</a>
 - <a href="#C7">Suppression des doublons</a>
 - <a href="#C8">Suppression des colonnes vides</a> 
  
**Partie 4 : Rapprochement des exports**
 - <a href="#C9">Renommage de la colonne id_web</a>
 - <a href="#C10">Jointure entre les 3 tables</a>
 
**Partie 5 : Chiffre d'affaires**
 - <a href="#C11">Calcul du CA par produit et le CA total en ligne</a>
 
**Partie 6 : Outliers**
 - <a href="#C12">Detection des outliers par la méthode du Z-score</a>
 - <a href="#C13">Detection des outliers par la méthode du IQR</a>
 - <a href="#C14">Visualisation graphique des outliers</a>

#<font color='indianred'>**Partie 1 : Importation des données**</font></a>

# <a name="C1"><font color='LightSeaGreen'>Importation des librairies</font></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy.stats as st

# <a name="C2"><font color='LightSeaGreen'>Importation des jeux de données : erp, liaison et web</font></a>

In [ ]:
#Ouvre les fichiers excel
!pip install openpyxl

In [ ]:
#importe les fichiers excel des données 
df_erp=pd.read_excel('../input/erp-datas/erp.xlsx')
pd.set_option('display.max_columns',None)
df_erp.head()

In [ ]:
df_liaison=pd.read_excel('../input/liaison-datas/liaison.xlsx')
df_liaison.head()

In [ ]:
df_web=pd.read_excel('../input/web-datas/web.xlsx')
pd.set_option('display.max_columns', 28)
df_web.head()

# <a name="C3"><font color='indianred'>Partie 2 : Analyse des jeux de données</font></a>

# <a name="C3"><font color='LightSeaGreen'>Affichage du nombre de lignes et colonnes des jeux de données</font></a>

In [ ]:
#nombre de lignes et colonnes
print('erp',df_erp.shape)
print('liaison',df_liaison.shape)
print('web',df_web.shape)

# <a name="C4"><font color='LightSeaGreen'>Analyse des valeurs manquantes</font></a>

In [ ]:
#Création d'une fonction calculant le pourcentage de valeurs manquantes 
#et affichant une heatmap pour visualiser les valeurs manquantes 
def val_manq(df):
    ligne,colonne=df.shape
    nb_cell=ligne*colonne
    nb_null=df.isnull().sum().sum()
    prct=nb_null *100/nb_cell
    print('Nombre de valeurs manquantes =',nb_null)
    print('Nombre de cellulles totales =',nb_cell)
    print('Pourcentage de valeurs manquantes = ',round(prct,2),'%') 
    print('\n')
    print('Pourcentage de valeurs manquantes avec le nombre de colonnes =\n',round((df.isna().sum()*100/df.shape[0]),2).value_counts(ascending=True))
    #Calcul des pourcentages de valeurs manquantes par variable(colonne)
    print('\n')
    print('Pourcentage de valeurs manquantes par variable/colonne =\n', 
          round((df.isna().sum()*100/df.shape[0]),2).sort_values(ascending=True))
    print('\n')
    plt.figure(figsize=(6,4))
    sns.heatmap(df.isna(), cbar=False)

In [ ]:
#Appel de la fonction val_manq 
#Affichage des pourcentages de valeurs manquantes des 3 jeux de données 
print('1-Table erp :')
val_manq(df_erp)
plt.title("Heatmap des valeurs manquantes erp")
print('---------------------------------------')
print('2-Table liaison :')
val_manq(df_liaison)
plt.title("Heatmap des valeurs manquantes liaison")
print('---------------------------------------')
print('3-Table web :')
val_manq(df_web)
plt.title("Heatmap des valeurs manquantes web")

Analyse des valeurs manquantes:
1. df_erp : il n'y a pas de valeurs manquantes
2. df_liaison : la colonne id_web contient 11% de valeurs manquantes

3. df_web : il y a 24% de valeurs manquantes
* 4 colonnes vides (100%)
* 3 colonnes sans valeurs manquantes (0%)
* 18 colonnes avec 5% de valeurs manquantes
* 3 colonnes avec 50% de valeurs manquantes


# <a name="C5"><font color='LightSeaGreen'>Analyse des types de données</font></a>

In [ ]:
#type des variables de la table erp
print(df_erp.dtypes)
print('\n')
#compte le nombre de variables par type
df_erp.dtypes.value_counts()

In [ ]:
#type des variables de la table liaison
print(df_liaison.dtypes)
print('\n')
#compte le nombre de variables par type
df_liaison.dtypes.value_counts()

In [ ]:
#type des variables de la table web
print(df_web.dtypes)
print('\n')
#compte le nombre de variables par type
df_web.dtypes.value_counts()

# <a name="C3"><font color='indianred'>Partie 3 : Nettoyage du jeu de données</font></a>

# <a name="C6"><font color='LightSeaGreen'>Unicité de la clé primaire</font></a>

In [ ]:
#Creation d'une fonction qui teste l'unicité d'une colonne
def cle_primaire(df,col):
    if len(df)==len(df[col].unique()):
        print('La variable', col, 'est une clé primaire')
    else: 
        print('La variable', col, 'n\'est pas une clé primaire')

In [ ]:
cle_primaire(df_erp,'product_id')
cle_primaire(df_web,'sku')

In [ ]:
#affiche le df dont le sku est null i.e qui n'ont pas de reference produit
df_websku = df_web[df_web['sku'].isnull()]
df_websku

Il y a 85 lignes sans sku.

In [ ]:
#affiche le df des produits qui ont un nom mais qui n'ont pas de reference produit (sku)
df_webskunotnull = df_websku[df_websku['post_name'].notnull()]
df_webskunotnull

Deux produits n'ont pas de sku.


# <a name="C7"><font color='LightSeaGreen'>Suppression des doublons</font></a>

In [ ]:
#Copie du dataframe web
df_web_db=df_web.copy()

### Création d'une fonction qui détecte les doublons

In [ ]:
#Fonction qui detecte les doublons
def detect_doublons(df):
    doub=df[df.duplicated()]
    print('------------------') 
    print('La table contient', len(doub), 'doublons.')
    print(doub.shape)
    print('\n') 
    print(doub)
    print('\n')

In [ ]:
print('------------------') 
print('Table erp')
detect_doublons(df_erp)
print('------------------') 
print('Table liaison')
detect_doublons(df_liaison)
print('------------------') 
print('Table web')
detect_doublons(df_web_db)

### Création d'une fonction qui supprime les doublons 


In [ ]:
#Création d'une fonction qui supprime les doublons et affiche la nouvelle heatmap sans les doublons
def supp_lignes(df):
    df=df.drop_duplicates(keep=False)
    print(df.shape)
    #print('Apres suppression des doublons, nous avons la heatmap suivante:')
    plt.figure(figsize=(6,4))
    sns.heatmap(df.isna(), cbar=False)    

In [ ]:
print('Table erp')
supp_lignes(df_erp)
print('Table liaison')
supp_lignes(df_liaison)
print('Table web')
supp_lignes(df_web_db)

In [ ]:
#Création du nouveau dataframe web contenant les lignes avec la valeur 'taxable' qui correspond au post du produit 
#alors que la 2eme ligne du meme produit correspond à la pièce jointe du post
df_webtax=df_web_db[(df_web_db['tax_status']=='taxable')&df_web_db['sku'].notnull()]
df_webtax.head()

In [ ]:
#1nouvelle heatmap du df web sans les lignes en double et les lignes des pieces jointes
sns.heatmap(df_webtax.isna(), cbar=False) 

# <a name="C8"><font color='LightSeaGreen'>Suppression des colonnes vides</font></a>

In [ ]:
#affiche la liste des colonnes vides du df web
#les 2 autres df erp et liaison n'ont pas de colonnes vides
print('Liste des colonnes vides:')
[col for col in df_webtax.columns if df_webtax[col].isnull().all()]

### Création d'une fonction qui supprime les colonnes vides

In [ ]:
#fonction qui compte, supprime les colonnes vides et affiche la nouvelle heatmap
def colonnes_vides(df): 
    l=[col for col in df.columns if df[col].isnull().all()]
    print('Nombre de colonnes vides =',len(l))
    for cv in l:
        df.pop(cv)
    print(df.shape)
    plt.figure(figsize=(6,4))
    sns.heatmap(df.isna(), cbar=False)

In [ ]:
#utilisation de la fonction qui compte et supprime les colonnes vides
colonnes_vides(df_webtax)

# <a name="C4"><font color='indianred'>Partie 4 : Rapprochement des exports</font></a>

# <a name="C9"><font color='LightSeaGreen'>Renommage de la colonne id_web</font></a>

In [ ]:
#Renommage de la colonne id_web de la table liaison correspondant à 'sku' dans la table web
df_liaison=df_liaison.rename(columns={'id_web':'sku'})
df_liaison.head(2)

# <a name="C10"><font color='LightSeaGreen'>Jointure des 3 tables</font></a>

## Jointure complète 

In [ ]:
#Jointure des 2 dataframes erp et liaison en faisant l'union complète 
df_erp_liaison=pd.merge(df_erp, df_liaison, left_on='product_id', right_on='product_id', how='outer')          
df_erp_liaison

In [ ]:
#Jointure des 2 dataframes erp_liaison et web en faisant l'union complète avec un indicateur 
#qui montre si la ligne appartient à df1 ou df2
#une colonne _merge se crée et indique both si la ligne appartient aux 2 df, 
#left_only si la ligne appartient au df de gauche
#Cette jointure lie toutes les lignes remplissant les cellules vides par une valeur nulle
df_final_outer=pd.merge(df_erp_liaison, df_webtax, left_on='sku', right_on='sku', how='outer', indicator=True)
df_final_outer.head()

In [ ]:
#on trouve 20 lignes dont les sku ne correspondent pas, avec comme valeur: _merge=left_only
#i.e une ligne appartient à la 1ere table et non à la 2eme table
df_final_outer[df_final_outer['_merge']!='both']

## Jointure inner

In [ ]:
#Cette jointure fait le lien interne avec product_id comme colonne commune 
df_erp_liaison_inner=pd.merge(df_erp, df_liaison, left_on='product_id', right_on='product_id', how='inner')          
df_erp_liaison_inner.head()

In [ ]:
#Cette jointure fait le lien interne avec sku comme colonne commune
df_final=pd.merge(df_erp_liaison_inner, df_webtax, left_on='sku', right_on='sku', how='inner', indicator=True)
df_final.head()

In [ ]:
#seulement les lignes communes ont ete fusionnées donc pas de nan 
df_final[df_final['_merge']!='both']

# <a name="C5"><font color='indianred'>Partie 5 : Chiffre d'affaires</font></a>

# <a name="C11"><font color='LightSeaGreen'>Calcul du CA par produit et du CA total</font></a>

In [ ]:
#ajout de la colonne 'CA_produit' qui calcule le CA par produit i.e calcule le CA pour chaque ligne
df_final['CA_produit']=df_final['price']*df_final['total_sales']
df_final.head()

### Total du Chiffre d'affaires

In [ ]:
#total du CA : somme de tous les CA par produit 
ca=sum(df_final['CA_produit'])
print('CA total:', ca, '€')

In [ ]:
#résumé des données 
df_final.describe()

# <a name="C12"><font color='LightSeaGreen'>Detection des outliers par la méthode du Z-score</font></a>

In [ ]:
#liste vide
outliers=[]

#fonction qui détecte, compte et liste les outliers
def liste_outliers(df):
    #seuil des valeurs aberrantes : +/- 2  
    threshold=2
    #moyenne
    mean=np.mean(df)
    #écart-type
    std=np.std(df)
    for i in df:
        zscore=(i-mean)/std
        if np.abs(zscore)>threshold:
            outliers.append(i)
    return outliers

#appel de la fonction 
l1=liste_outliers(df_final['price'])
print('Nombre d\'outliers :', len(l1))
print('\n') 
print('Liste des outliers : ', l1)

# <a name="C13"><font color='LightSeaGreen'>Detection des outliers par la méthode du IQR</font></a>

In [ ]:
#détecte les outliers et les filtre pour avoir un nouveau dataframe sans outliers
#1er quartile
q1 = df_final['price'].quantile(0.25)
#3eme quartile
q3 = df_final['price'].quantile(0.75)
#écart interquatile
iqr = q3 - q1
print('Ecart interquatile =', round(iqr,2))
seuil_min = q1 -(1.5 * iqr) 
seuil_max = q3 +(1.5 * iqr)
print('Seuil minimum =', round(seuil_min,2))
print('Seuil maximum =', round(seuil_max,2))
#affiche le dataframe final sans outliers
df_out = df_final[(df_final['price'] > seuil_min) & (df_final['price'] < seuil_max)]
#trie par ordre croissant les donnees du df web sans outliers via la colonne 'price' 
df_out.sort_values('price', ascending=True).head()

In [ ]:
#dataframe des outliers
dfo = df_final[(df_final['price'] < seuil_min) | (df_final['price'] > seuil_max)]
#liste des outliers
liste_out=(dfo['price'].tolist())
print('Nombre doutliers:', len(liste_out))
print('\n')
print('Liste des outliers :', liste_out)
print('\n')
#affichage des outliers 'price' avec les colonnes 'stock_quantity','total_sales', 'CA_produit'
dfo[['price', 'stock_quantity','total_sales', 'CA_produit']].head()

### <a name="C14"><font color='LightSeaGreen'>Visualisation graphique des outliers</font></a>

In [ ]:
#Box Plot / Boîte à moustaches
#montre les valeurs extrêmes et la symétrie, la dispersion ou la centralité de la distribution des valeurs 
#associées à la variable 'price'
#pas de symétrie dans la distribution
df_final.boxplot(column='price', vert=False)

In [ ]:
#histogramme pour la variable quantitative continue price qui montre les valeurs extremes à droite 
#l'histogramme est étalé à droite et presente donc une asymétrie 
df_final['price'].hist(density=True, bins=100)
plt.show()

In [ ]:
#Diagramme de dispersion / Nuage de points mettant en relief la relation entre les 2 variables price et total_sales
#montre la concentration de la plupart des prix à gauche et les valeurs extremes à droite
#plus les prix sont élevés, moins il y a de ventes des produits
df_final.plot.scatter( x="price", y="total_sales", s = 50)